<a href="https://colab.research.google.com/github/vedantkathe846/code_gladiators/blob/main/TGLR's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from spacy.lang.en import English
import pandas as pd
import numpy as np
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy import displacy
from IPython.display import clear_output
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier
import copy

In [9]:
data=pd.read_csv('/content/drive/MyDrive/News/news.tsv',header=None,sep='\t')
data.columns=['News ID',
"Category",
"SubCategory",
"Title",
"Abstract",
"URL",
"Title Entities",
"Abstract Entities "]
# Copying the dataset
df=copy.deepcopy(data)

In [10]:
data.head()

,News ID,Category,SubCategory,Title,Abstract,URL,Title Entities,Abstract Entities
0,N55528,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N19639,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N61837,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
3,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."
4,N38324,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://assets.msn.com/labs/mind/AAAKEkt.html,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."


In [11]:
nlp = English()
texts=data['Title'].values

#  "nlp" Object is used to create documents with linguistic annotations.
end_text=[]
# Create list of word tokens
for text in texts:
    my_doc = nlp(text)
    token_list = []
    for token in my_doc:
        token_list.append(token.text)
    end_text.append(token_list)
data['Title']=end_text

In [12]:
# Having a look at the tokenized Titles
data['Title']

0        [The, Brands, Queen, Elizabeth, ,, Prince, Cha...
1                     [50, Worst, Habits, For, Belly, Fat]
2        [The, Cost, of, Trump, 's, Aid, Freeze, in, th...
3        [I, Was, An, NBA, Wife, ., Here, 's, How, It, ...
4        [How, to, Get, Rid, of, Skin, Tags, ,, Accordi...
                               ...                        
51277    [Adapting, ,, Learning, And, Soul, Searching, ...
51278    [Family, says, 13-year, -, old, Broadway, star...
51279    [St., Dominic, soccer, player, tries, to, kick...
51280                  [How, the, Sounders, won, MLS, Cup]
51281             [Best, Sports, Car, Deals, for, October]
Name: Title, Length: 51282, dtype: object

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
# Making a function to lemmatize all the words
lemmatizer = WordNetLemmatizer() 
def lemmatize_all(data,name):
    arr=data[name]
    a=[]
    for i in arr:
        b=[]
        for j in i:
            x=lemmatizer.lemmatize(j,pos='a')
            x=lemmatizer.lemmatize(x)
            b.append(x)
        a.append(b)
    data[name]=a

In [15]:
# Titles after lemmatizing 
lemmatize_all(data,'Title')
data['Title']

0        [The, Brands, Queen, Elizabeth, ,, Prince, Cha...
1                     [50, Worst, Habits, For, Belly, Fat]
2        [The, Cost, of, Trump, 's, Aid, Freeze, in, th...
3        [I, Was, An, NBA, Wife, ., Here, 's, How, It, ...
4        [How, to, Get, Rid, of, Skin, Tags, ,, Accordi...
                               ...                        
51277    [Adapting, ,, Learning, And, Soul, Searching, ...
51278    [Family, say, 13-year, -, old, Broadway, star,...
51279    [St., Dominic, soccer, player, try, to, kick, ...
51280                  [How, the, Sounders, won, MLS, Cup]
51281             [Best, Sports, Car, Deals, for, October]
Name: Title, Length: 51282, dtype: object

In [16]:
nlp = spacy.load('en')
news=nlp(df['Title'][51278])
entities=[(i, i.label_, i.label) for i in news.ents]
entities

[(Broadway, 'FAC', 9191306739292312949)]

In [17]:
  displacy.render(news, style = "ent",jupyter = True)

In [18]:
news=df['Title'][51278]
print('The news headline is :',news)
news=nlp(news)
displacy.render(news, style="dep", jupyter= True)

The news headline is : Family says 13-year-old Broadway star died from massive asthma attack


In [19]:
def make_to_sentence(data,name):
    x=[]
    for i in data[name].values:
        p=''
        for j in i:
            p+=j.lower()+' '
        x.append(p)
    data[name]=x

In [20]:
make_to_sentence(data,'Title')

In [21]:
X=data['Title'].values
y=data['Category'].values

In [22]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,shuffle=True, random_state=42)

In [23]:
classifier = LogisticRegression()

In [24]:
tfidf_vector = TfidfVectorizer()

In [25]:
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

In [26]:
pipe.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [27]:
# Checking The score of the model
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.699657291420468
Logistic Regression Precision: 0.7015827983414743
Logistic Regression Recall: 0.699657291420468


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
xgb=XGBClassifier()

In [29]:
pipe_xgb = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', xgb)])

In [30]:
pipe_xgb.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', XGBClassifier(objective='multi:softprob'))])

In [31]:
# Checking The score of the model
predicted = pipe_xgb.predict(X_test)

# Model Accuracy
print("XGBClassifier Accuracy:",metrics.accuracy_score(y_test, predicted))
print("XGBClassifier Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("XGBClassifier Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

XGBClassifier Accuracy: 0.5645237532498227
XGBClassifier Precision: 0.6266002285251465
XGBClassifier Recall: 0.5645237532498227


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [32]:
lgbm=LGBMClassifier()

In [33]:
pipe_lgb = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', lgbm)])

In [34]:
pipe_lgb.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LGBMClassifier())])

In [35]:
# Checking The score of the model
predicted = pipe_lgb.predict(X_test)

# Model Accuracy
print("LGBMClassifer Accuracy:",metrics.accuracy_score(y_test, predicted))
print("LGBMClassifer Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("LGBMClassifer Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

LGBMClassifer Accuracy: 0.6477783030016545
LGBMClassifer Precision: 0.6458766549952709
LGBMClassifer Recall: 0.6477783030016545


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
rf=RandomForestClassifier()

In [37]:
pipe_rf = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', rf)])

In [38]:
pipe_rf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [39]:
# Checking The score of the model
predicted = pipe_rf.predict(X_test)

# Model Accuracy
print("Random Forest Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Random Forest Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Random Forest Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

Random Forest Accuracy: 0.6373788702434413
Random Forest Precision: 0.6448710877495412
Random Forest Recall: 0.6373788702434413


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
knn_model = KNeighborsClassifier()

In [41]:
pipe_knn_model = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', knn_model)])

In [42]:
pipe_knn_model.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', KNeighborsClassifier())])

In [43]:
# Checking The score of the model
predicted = pipe_knn_model.predict(X_test)

# Model Accuracy
print("KNN Accuracy:",metrics.accuracy_score(y_test, predicted))
print("KNN Forest Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("KNN Forest Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

KNN Accuracy: 0.6741904987000709
KNN Forest Precision: 0.6614745668069837
KNN Forest Recall: 0.6741904987000709


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
dt = DecisionTreeClassifier()

In [45]:
pipe_dt = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', dt)])

In [46]:
pipe_dt.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', DecisionTreeClassifier())])

In [47]:
# Checking The score of the model
predicted = pipe_dt.predict(X_test)

# Model Accuracy
print("Decision Tree Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Decision Tree Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Decision Tree Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

Decision Tree Accuracy: 0.5278303001654455
Decision Tree Precision: 0.5105946058008303
Decision Tree Recall: 0.5278303001654455


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
Ada = AdaBoostClassifier()

In [49]:
pipe_Ada = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', Ada)])

In [50]:
pipe_Ada.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', AdaBoostClassifier())])

In [51]:
# Checking The score of the model
predicted = pipe_Ada.predict(X_test)

# Model Accuracy
print("Ada Boost Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Ada Boost Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Ada Boost Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

Ada Boost Accuracy: 0.31050579059324035
Ada Boost Precision: 0.32414972225685124
Ada Boost Recall: 0.31050579059324035


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
GB = GradientBoostingClassifier()

In [53]:
pipe_GB = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', GB)])

In [54]:
pipe_GB.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', GradientBoostingClassifier())])

In [55]:
# Checking The score of the model
predicted = pipe_GB.predict(X_test)

# Model Accuracy
print("GB Boost Accuracy:",metrics.accuracy_score(y_test, predicted))
print("GB Boost Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("GB Boost Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

GB Boost Accuracy: 0.612148428267549
GB Boost Precision: 0.6314604851039686
GB Boost Recall: 0.612148428267549


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
clf = RidgeClassifier()

In [57]:
pipe_clf = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', clf)])

In [58]:
pipe_clf.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', RidgeClassifier())])

In [59]:
# Checking The score of the model
predicted = pipe_clf.predict(X_test)

# Model Accuracy
print("Ridge Classifier Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Ridge Classifier Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Ridge Classifier Recall:",metrics.recall_score(y_test, predicted,average='weighted'))

Ridge Classifier Accuracy: 0.7277830300165445
Ridge Classifier Precision: 0.7109032667604461
Ridge Classifier Recall: 0.7277830300165445


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
import pickle

In [65]:
outfile = open("TGLR.pkl", 'wb')

In [66]:
pickle.dump(data , outfile)
outfile.close

<function BufferedWriter.close>